## Preprocess training data to BERT's format. 
### we will get train.tsv and dev.tsv

In [2]:
import random
random.seed = 16
import pandas as pd
import jieba
from collections import Counter
import numpy as np
import os
import pickle
import re
from hanziconv import HanziConv

import ended...


In [3]:
print("read csv started...")
data = pd.read_csv("ai_challenger_sentiment_analysis_trainingset_20180816/sentiment_analysis_trainingset.csv")
data_validation = pd.read_csv("ai_challenger_sentiment_analysis_validationset_20180816/sentiment_analysis_validationset.csv")
data_test_old = pd.read_csv("ai_challenger_sentiment_analysis_testa_20180816/sentiment_analysis_testa.csv")
# data_test = pd.read_csv("ai_challenger_sentimetn_analysis_testb_20180816/sentiment_analysis_testb.csv")

print("training.shape:",data.shape)
print("data_validation.shape:",data_validation.shape)
print("data_test_old.shape:",data_test_old.shape)
# print("data_test.shape:",data_test.shape)
print("read csv ended...")

read csv started...
training.shape: (105000, 22)
data_validation.shape: (15000, 22)
data_test_old.shape: (15000, 22)
read csv ended...


In [17]:
def get_sentiment_analysis_labels(row):
    # 1)location
    location_traffic_convenience = row['location_traffic_convenience']               
    location_distance_from_business_district= row['location_distance_from_business_district'] 
    location_easy_to_find  = row['location_easy_to_find']  
    # 2)service
    service_wait_time  = row['service_wait_time']                         
    service_waiters_attitude = row['service_waiters_attitude']                     
    service_parking_convenience = row['service_parking_convenience']                 
    service_serving_speed  = row['service_serving_speed']  
    # 3)price
    price_level         = row['price_level']                        
    price_cost_effective = row['price_cost_effective']                          
    price_discount  = row['price_discount']     
    # 4)environment
    environment_decoration  = row['environment_decoration']                   
    environment_noise   = row['environment_noise']                        
    environment_space   = row['environment_space']                         
    environment_cleaness     = row['environment_cleaness']  
    # 5)dish
    dish_portion   = row['dish_portion']                              
    dish_taste =row['dish_taste']                                   
    dish_look  = row['dish_look']                                  
    dish_recommendation = row['dish_recommendation']   
    # 6)other
    others_overall_experience  = row['others_overall_experience']                    
    others_willing_to_consume_again   = row['others_willing_to_consume_again']   
    
    label_list=[]
    label_list=[location_traffic_convenience,location_distance_from_business_district,location_easy_to_find, # location
                  service_wait_time,service_waiters_attitude,service_parking_convenience,service_serving_speed, # service
               price_level,price_cost_effective,price_discount, # price
               environment_decoration,environment_noise,environment_space,environment_cleaness, # environment
               dish_portion,dish_taste,dish_look,dish_recommendation, # dish
               others_overall_experience,others_willing_to_consume_again] # other
    label_list=[str(i)+"_"+str(label_list[i]) for i  in range(len(label_list))]
    return label_list 

In [18]:
def convet_to_one_hot(label_list,num_classes=80):
    new_label_list=[0 for i in range(num_classes)]
    for label in label_list:
        new_label_list[label]=1
    return new_label_list

label_list=[0, 4, 8, 12, 19, 20, 24, 28, 32, 39, 43, 47, 51, 55, 59, 62, 64, 68, 75, 76]
label_list_one_hot=convet_to_one_hot(label_list)
print("label_list_one_hot:",label_list_one_hot)

label_list_one_hot: [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]


In [20]:
data_traininig_small=data.sample(frac=1.0).sample(frac=1.0)
data_validation_small=data_validation.sample(frac=1.0).sample(frac=1.0)
data_test_small=data_test_old #[0:1000]
# data_test2_small=data_test #[0:1000]

print("data_traininig_small.shape:",data_traininig_small.shape)
print("data_validation_small.shape:",data_validation_small.shape)
print("data_test_small.shape:",data_test_small.shape)
# print("data_test2_small.shape:",data_test2_small.shape)

data_traininig_small.shape: (105000, 22)
data_validation_small.shape: (15000, 22)
data_test_small.shape: (15000, 22)


In [16]:
# generate training data by following data format of bert

def process_data(data,target_file):
    target_object=open(target_file,'w', encoding='utf-8')
    print(data.shape)
    for index, row in data.iterrows():
        sentiment_label_list=get_sentiment_analysis_labels(row) 
        content = row['content'].strip('"').strip()
        
        content = per_content(content)
        
        strings=",".join(sentiment_label_list)+"\t"+content+"\n"
        target_object.write(strings)
        if index%50000==0:print(strings)
    target_object.close()
    

In [12]:
import json
ig_char_p = 'dataset/ignore_unk.txt'
with open(ig_char_p) as f:
    ignore_char = json.load(f)
sep = ['.', ',', '，', '!', '！', '。', '?', '?', ';', '；', ':', '：']

In [23]:
def per_content(content):
    content = content.replace('★★★★★', '非常好')
    content = content.replace('★★★★☆', '好')
    content = content.replace('★★★☆☆', '一般')
    content = content.replace('★★☆☆☆', '差')
    content = content.replace('★☆☆☆☆', '很差')

    content = content.replace('★★★★', '好')
    content = content.replace('★★★', '一般')
    content = content.replace('★★', '差')

    content = content.replace('⭐️⭐️⭐️⭐️⭐️', '非常好')
    content = content.replace('⭐️⭐️⭐️⭐️', '好')
    content = content.replace('⭐️⭐️⭐️', '一般')
    content = content.replace('⭐️⭐️', '差')
    content = content.lower()
    content = content.replace("\n\r","\n" )  
    contents = content.split('\n')
    tmps = []
    for x in contents:
        tmps.append(x.replace(' ', '，'))

    content = '。'.join(tmps)
    content = content.replace("\r","。").replace(".","。").replace(",","，")

    content=[content[i] for i in range(len(content)) if content[i].strip() and content[i]!="\"" ]
    content="".join(content)
    content=re.sub('''"''','',content)


    content = HanziConv.toSimplified(content) 
    content_token = list(jieba.cut(content))
    tokens = []
    pre = None
    for x in content_token:
        if x in ignore_char:
            continue
        if x in sep and pre == x:
            continue
        pre = x
        tokens.append(x)
    content = ''.join(tokens)
                            
    return content

In [15]:
s = """4人同行 点了10个小吃
榴莲酥 榴莲味道不足 松软 奶味浓
虾饺 好吃 两颗大虾仁
皮蛋粥 皮蛋多 但是一般 挺稠的
奶黄包 很好吃 真的是蛋黄和奶 而且真的是流沙
叉烧包 面香
鲜虾烧卖 好吃 外面的黄色皮看着让人特别有食欲
云吞面 云吞分量足 但是汤头不是很好喝 而且云吞的馅儿不知为何感觉不是很新鲜
鲍汁腐皮卷 没怎么吃 味道倒是不错
排骨 味道不错 不算很腻 但是油确实微多
鲜虾锅贴 确实今天吃了很多虾 这个很酥脆，里头的虾也很好吃

刚好有优惠券，所以4个人花了100不到【这个优惠券只能在1层用，5层用不了】

原价大概人均50

服务一般，上菜速度倒是很快，人挺多，坐在沙发上感觉很舒服"""
per_content(s)

'人同行,点了个小吃。榴莲酥,榴莲味道不足,松软,奶味浓。虾饺,好吃,两颗大虾仁。皮蛋粥,皮蛋多,但是一般,挺稠的。奶黄包,很好吃,真的是蛋黄和奶,而且真的是流沙。叉烧包,面香。鲜虾烧卖,好吃,外面的黄色皮看着让人特别有食欲。云吞面,云吞分量足,但是汤头不是很好喝,而且云吞的馅儿不知为何感觉不是很新鲜。鲍汁腐皮卷,没怎么吃,味道倒是不错。排骨,味道不错,不算很腻,但是油确实微多。鲜虾锅贴,确实今天吃了很多虾,这个很酥脆，里头的虾也很好吃。刚好有优惠券，所以个人花了不到【这个优惠券只能在层用，层用不了】。原价大概人均。服务一般，上菜速度倒是很快，人挺多，坐在沙发上感觉很舒服'

In [24]:
target_file_valid='bert_data/dev.tsv'
process_data(data_validation_small,target_file_valid)
print("process_data.valid.ended.",len(data_validation_small))
#X_test,_=process_data(data_test_small,data_type='test')
#print("process_data.ended...")


(15000, 22)
0_-2,1_-2,2_-2,3_0,4_-2,5_-2,6_-2,7_1,8_-2,9_-2,10_-2,11_-2,12_-2,13_-2,14_-2,15_0,16_-2,17_-2,18_1,19_0	哎，想当年来畲山的时候，啥都没有，三品香算镇上最大看起来最像样的饭店了。菜品多，有点太多，感觉啥都有，杂都不足以形容。随便点些，居然口味什么的都好还可以，价钱自然是便宜当震惊。元宝虾和椒盐九肚鱼都不错吃。不过近来几次么，味道明显没以前好了。冷餐里面一个凉拌海带丝还可以，酸酸甜甜的。镇上也有了些别的大点的饭店，所以不是每次必来了。对了，这家的生意一如既往的超级好，不定位基本吃不到。不过畲山这边的人吃晚饭很早的，所以稍微晚点去就很空了。

process_data.valid.ended. 15000


In [25]:
target_file_train='bert_data/train.tsv'
process_data(data_traininig_small,target_file_train)
print("process_data.train.ended.",len(data_traininig_small))

(105000, 22)
0_1,1_1,2_-2,3_-2,4_1,5_1,6_-2,7_-2,8_-2,9_1,10_1,11_1,12_1,13_1,14_1,15_1,16_-2,17_-2,18_1,19_1	非常喜欢去新开的有特色的店品尝，有幸被大众点评网同城聚会抽到黄金替补免费品鉴的机会，我感到十分幸运！红不让台湾特色餐厅位于李沧万达金街号入口扶梯旁，交通便利，停车方便。店面小资，适合举行小型派对。走近店门，服务员立刻开门迎接并且热情的打招唿。进入店面，环顾四周，鹅黄的灯光下透露出温馨慵懒的气氛，店家匠心独居，屋内的摆设典雅质朴。我们大家围坐在一起开始品尝台湾特色美食，前菜是鸡排沙拉，鸡排外焦里嫩，沙拉酸爽开胃陪的饮品是本店招牌—寒天奶栋，口感丝化。在我们的玩闹嘻戏中，硬菜短上来了——招牌全垒打拼盘，样子可爱，味道独特，被大家迅雷不及掩耳盗铃之势一扫而光。紧接着——等一个人小火锅，西班牙海鲜焗饭，黑椒牛柳焗饭海鲜焗烤嫩鸡排上桌了，食材琳琅满目，令人目不暇接。人多力量大，盘子渐渐见底了，新菜品——普罗旺斯田园披萨，台湾卤肉饭，秘制鸡腿饭，招牌巨蛋烧，阳光手打面，超级霸大热狗震撼来袭。这里特别提一下普罗旺斯田园披萨，顶上配料五颜六色，芝士浓郁，饼胚酥脆，搭配沙拉和洋葱圈口味绝佳。还有在卤肉饭，鸡腿饭，手打面扮演重要角色的酱料，里面居然搭配了小鱼干，补钙又美味。环境搭配合理，既有闺密情侣说悄悄话的私密空间，又有三五好友畅饮阔聊的长桌吧台，气氛轻松惬意，互不打扰。红不让台湾特色餐厅绝对是小资的你餐饮休闲，畅聊发呆的绝佳选择，千万不要错过！

0_-2,1_-2,2_-2,3_-2,4_1,5_-2,6_-2,7_-2,8_-2,9_1,10_-2,11_-2,12_-2,13_-2,14_-2,15_-2,16_1,17_-2,18_1,19_-2	吼吼吼，萌死人的棒棒糖，中了大众点评的霸王餐，太可爱了。一直就好奇这个棒棒糖是怎么个东西，大众点评给了我这个土老冒一个见识的机会。看介绍棒棒糖是用德国糖做的，不会很甜，中间的照片是糯米的，能食用，真是太高端大气上档次了，还可以买蝴蝶结扎口，送人可以买礼盒。我是先打的卖家电话，加了微信，给卖家传的照片。等了几天，卖家就告诉我可以取货了，去大官屯那取的。虽然连卖家的面都没见到，但是还是谢谢卖家送我这么可爱的东西，太喜欢